In [83]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.model_selection import train_test_split
from itertools import permutations, product
import functools
from sklearn.metrics import accuracy_score, confusion_matrix
import tqdm
from numba import njit,jit
import numpy as np
import pickle
from os import path

from warnings import simplefilter
from collections import OrderedDict

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)


In [4]:
df = pd.read_csv('virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

# make both have the same amount of samples..?

thresh = len(df)
# df2.drop(df2.index[thresh:len(df2.index)], inplace=True)

dfother = pd.DataFrame()
it = len(df)+1
kmer = 4

print(len(df))
print(len(df2))
# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']

    it+=1


mergedDf = pd.concat([df, dfother], axis=0)


1236
8192


In [5]:
# print(df.shape)
# print(df2.shape)

print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [6]:
kmer = 4
s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

del s
print(permset)
print(len(permset))


{'atac', 'ccta', 'aaca', 'aaat', 'aggc', 'tcgc', 'ttaa', 'taga', 'accc', 'tcca', 'cggt', 'ctcc', 'tata', 'gttt', 'gtag', 'gaat', 'cgct', 'aatt', 'cacc', 'tact', 'accg', 'attt', 'aact', 'taca', 'tttg', 'ccct', 'tctg', 'cgtc', 'ctga', 'cgga', 'cctc', 'ccgg', 'cgcc', 'tctc', 'ggta', 'ccca', 'gtta', 'catc', 'cgca', 'caat', 'tcag', 'cata', 'gaga', 'ttgt', 'gtaa', 'agcg', 'agct', 'cgcg', 'tcgg', 'gatc', 'tgtg', 'aata', 'cagc', 'ctag', 'gcat', 'acat', 'ttac', 'cctg', 'acac', 'atag', 'tggt', 'gcgt', 'cact', 'cttc', 'ctaa', 'agtt', 'tttt', 'cccg', 'ttgc', 'gcag', 'tgcc', 'agcc', 'cgat', 'ggct', 'gatg', 'gttg', 'ccga', 'tgta', 'atct', 'ccag', 'ccgt', 'ctta', 'atcg', 'ggcc', 'atta', 'gacg', 'tacg', 'tgat', 'gcca', 'catt', 'ctgc', 'aagt', 'aaga', 'gccc', 'acgc', 'caaa', 'ggtc', 'ggat', 'tagt', 'gcac', 'atca', 'aggg', 'cccc', 'agac', 'tatc', 'cgta', 'gact', 'agtg', 'tcga', 'tgtt', 'gctg', 'agaa', 'cagt', 'gcga', 'tgag', 'agca', 'ttct', 'acga', 'gcta', 'aagg', 'cgtg', 'tgac', 'ctat', 'atgc', 'cgac',

In [7]:
def ComputeKmerVector(sequence,k,KmerCount):
    n = len(sequence)
    # find number of occurrences of certain kmers, then 
    for j in range(0,n-k+1):
        kmer = sequence[j:(j+k)]
        if kmer in KmerCount:
            KmerCount[kmer] += 1
        else:
            continue

    KmerVec = OrderedDict()

    
    for m in KmerCount:
        KmerVec[m] = float(KmerCount[m])/float(n-k+1)
    Vec = list(KmerVec.values())
    
    return Vec
def queryKmer(name,kmer):
    
    KmerCount = OrderedDict()
    s = itertools.product('acgt',repeat = kmer)
    sl = list(s)
    for i in range(0,len(sl)):
        a0 = str(sl[i])
        a1 = a0.replace("(","")
        a2 = a1.replace(")","")
        a3 = a2.replace(",","")
        a4 = a3.replace("'","")
        a4 = a4.replace(" ", "")
        KmerCount[a4] = 0
    zero = [0 for _ in range(len(sl))]
    
    total_lst = list()
    for line in name:
        tt = line.strip('\n').split(' ')
        Seq = tt[1].upper()
        KmerCount = OrderedDict(zip(KmerCount.keys(),zero))
        temp = " ".join(map(str,ComputeKmerVector(Seq,kmer,KmerCount)))
        temp = tt[0]+' '+temp
        IVL = temp.split(' ')
        total_lst.append(IVL)
    ID_VEC_LABEL = pd.DataFrame(total_lst)
    return ID_VEC_LABEL

In [29]:
dna = mergedDf['DNA Sequence']
Y = mergedDf['isZoonotic']

kmer = 4

def resetkmerdict()->OrderedDict:
    kmerdict = OrderedDict()
    for i in permset:
        kmerdict[i]=0
    return kmerdict

def assign_kmers_to_dict(row):
    kmerdict=resetkmerdict()
    st = row[2]
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict


asdf=0
l = []

for row in tqdm.tqdm(mergedDf.itertuples()):
    l.append(assign_kmers_to_dict(row))

finalkmerdict=pd.DataFrame(l)
print(finalkmerdict)

print("finished")

9428it [01:18, 120.68it/s]


      atac  ccta  aaca  aaat  aggc  tcgc  ttaa  taga  accc  tcca  ...  gcgc  \
0       49    33    79    63    26     3    56    40    21    28  ...     5   
1       44    34    59    68    26    11    47    32    27    27  ...     7   
2       48    37    71    43    61    31    18    30    51    59  ...    42   
3       43    43    61    48    35    10    41    39    33    32  ...    10   
4       26    22    45    39    47     8    30    22    47    38  ...    12   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9423     9     8    13    14    16     4     4     6     9    19  ...     5   
9424   143    67   193   289    77    76   204   125    62   140  ...   106   
9425    22    21    49    55    29     9    30    31    15    29  ...    10   
9426    52    38    86    70    53    12    42    51    58    31  ...    14   
9427    19    40    58    39    57    22     5    35    53    65  ...    15   

      ttag  ccgc  ttcg  tgaa  tccc  taag  ttat  cac

In [9]:
# mergedDf.drop(mergedDf.index[100:mergedDf.shape[0]], inplace=True)

for i in range(3, mergedDf.shape[1]-1):
    for j in range(0, 100):
        print(mergedDf.head().loc[j])
        str(mergedDf.head().loc[j][mergedDf.columns[i]]) != "nan" and print(mergedDf.head().loc[0][mergedDf.columns[i]])
# print(len(mergedDf))


In [38]:
mergedDf.fillna(0, inplace=True)

X = finalkmerdict
Y = mergedDf['isZoonotic']

print(X)
print(Y)

      atac  ccta  aaca  aaat  aggc  tcgc  ttaa  taga  accc  tcca  ...  gcgc  \
0       49    33    79    63    26     3    56    40    21    28  ...     5   
1       44    34    59    68    26    11    47    32    27    27  ...     7   
2       48    37    71    43    61    31    18    30    51    59  ...    42   
3       43    43    61    48    35    10    41    39    33    32  ...    10   
4       26    22    45    39    47     8    30    22    47    38  ...    12   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9423     9     8    13    14    16     4     4     6     9    19  ...     5   
9424   143    67   193   289    77    76   204   125    62   140  ...   106   
9425    22    21    49    55    29     9    30    31    15    29  ...    10   
9426    52    38    86    70    53    12    42    51    58    31  ...    14   
9427    19    40    58    39    57    22     5    35    53    65  ...    15   

      ttag  ccgc  ttcg  tgaa  tccc  taag  ttat  cac

In [11]:
print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

zz = X_train.head()

# print([mergedDf[x].sum() for x in mergedDf if x != 'ID' and x != 'DNA Sequence' and x != 'isZoonotic'])
# for i in range(10):
#     print(i)
#     print(mergedDf[mergedDf.isin([i]).any(axis=1)])
# print(mergedDf.loc[mergedDf[mergedDf.columns] == 1])

In [44]:
model = LogisticRegression(
    multi_class="multinomial", max_iter=1000,
    fit_intercept=False, tol=0.001, solver='saga', random_state=42)
model.fit(X_train, y_train)

LogisticRegression(fit_intercept=False, max_iter=1000,
                   multi_class='multinomial', random_state=42, solver='saga',
                   tol=0.001)

In [46]:
# all 0s for some reason
predictions = model.predict(X_test)
asdf = [x for x in predictions if x == 1]
print(asdf)
print(accuracy_score(y_test, predictions))

[]
0.8685047720042418


In [59]:
knntest = BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors = 1, n_jobs = 1), n_estimators = 10, n_jobs = 1)

knntest.fit(X_train, y_train)
# knn = KNeighborsClassifier(n_neighbors=5)

BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_jobs=1,
                                                              n_neighbors=1),
                          n_jobs=1)

In [60]:
predictions = knntest.predict(X_test)
print(predictions[:10])
print(accuracy_score(y_test, predictions))

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0.9236479321314952


In [74]:
randforest = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features="auto"))

randforest.fit(X_train, y_train)

/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features='auto'))

In [86]:
predictions = randforest.predict(X_test)
print(predictions[:100])
y_test_list = [x for x in y_test[:100]]

if not path.exists("randomforest.pkl"):
    pickle.dump(randforest, open('randomforest.pkl', 'wb'))
else:
    pickled_model = pickle.load(open('randomforest.pkl', 'rb'))
    if accuracy_score(y_test, predictions) > accuracy_score(pickled_model.predict(X_test), predictions):
        print("accuracy higher, updating model pickle")
        pickle.dump(randforest, open('randomforest.pkl', 'wb'))
    else:
        print("accuracy same/lower")



for i in range(100):
    print(predictions[i], "vs", y_test_list[i])

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
accuracy same/lower
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 1.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 1.0
1.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 1.0
1.0 vs 1.0
0.0 vs 0.0
1.0 vs 1.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
1.0 vs 1.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0 vs 0.0
0.0

/var/folders/vf/6r913rfx607_3y64x5l2zw4r0000gn/T/ipykernel_75427/3030018759.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test_list = [x for x in y_test[:100]]
